# Анализ текущего уровня NPS телекоммуникационной компании среди клиентов из России

Заказчик данного исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:

9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Цель исследования — ответить на следующие вопросы:

Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

Какие группы пользователей наиболее лояльны к сервису? Какие менее?

Какой общий NPS среди всех опрошенных?

Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

In [1]:
# импортируем библиотеки
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
# SQL запрос
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'Не определен'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""

In [4]:
# создаём датафрейм по данным запроса
df = pd.read_sql(query, engine)
# выводим первые 10 строк датафрейма
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Cторонник
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Cторонник
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Cторонник
3,A004TB,4190,Старый,44.0,Женщина,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Cторонник
4,A004XT,1163,Старый,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Cторонник
5,A005O0,5501,Старый,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,Критик
6,A0061R,1236,Старый,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,Cторонник
7,A009KS,313,Новый,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,Cторонник
8,A00AES,3238,Старый,36.0,Женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,Cторонник
9,A00F70,4479,Старый,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,Cторонник


In [5]:
# выводим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


В таблице 502493 строки и 14 столбцов:

- user_id — идентификатор клиента
- lt_day — количество дней «жизни» клиента
- is_new — поле хранит информацию о том, является ли клиент новым
- age — возраст
- gender_segment — пол
- os_name — тип операционной системы
- cpe_type_name — тип устройства
- country — страна проживания
- city — город проживания
- age_segment — возрастной сегмент
- traffic_segment — сегмент по объёму потребляемого трафика
- lifetime_segment — сегмент по количеству дней «жизни»
- nps_score — оценка клиента в NPS-опросе
- nps_group — поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [6]:
# получившуюся таблицу сохраняем как .csv файл.
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Ссылка на дашборд:

https://public.tableau.com/views/_2_16862592983570/NPS_1_1?:language=en-US&:display_count=n&:origin=viz_share_link

# Ссылка на презентацию:

https://disk.yandex.ru/i/KUjmjjTNfLGTOA